In [1]:
from pybaseball import batting_stats, pitching_stats, statcast_batter, playerid_lookup
import pandas as pd

In [2]:
batting_data = batting_stats(2024, qual=50)
batting_data.head()

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,15640,2024,Aaron Judge,NYY,32,158,559,704,180,85,...,117.5,238,0.609,391,0.146,0.267,0.310,0.723,0.479,11.5
6,25764,2024,Bobby Witt Jr.,KCR,24,161,636,709,211,123,...,116.9,259,0.481,538,0.138,0.236,0.315,0.577,0.407,10.4
2,19755,2024,Shohei Ohtani,LAD,29,159,636,731,197,98,...,119.2,287,0.595,482,0.144,0.269,0.314,0.660,0.442,9.2
3,20123,2024,Juan Soto,NYY,25,157,576,713,166,90,...,115.7,262,0.568,461,0.189,0.263,0.316,0.646,0.462,7.9
19,26289,2024,Gunnar Henderson,BAL,23,159,630,719,177,102,...,113.1,256,0.539,475,0.167,0.265,0.283,0.492,0.374,8.2


In [3]:
batting_data.fillna(0, inplace=True)

In [4]:
batter_name_lookup = batting_data[['IDfg', 'Name']]

batter_versus_pitch_type = batting_data[['IDfg', 'wFB', 'wSL', 'wCT', 'wCB', 'wCH', 'wSF', 'wKN']]

batter_hit_stats = batting_data[['IDfg', 'AVG+', 'OBP+', 'BABIP+', 'SLG+', 'BB%+', 'HardHit%']]

batter_name_lookup.head()

,IDfg,Name
0,15640,Aaron Judge
6,25764,Bobby Witt Jr.
2,19755,Shohei Ohtani
3,20123,Juan Soto
19,26289,Gunnar Henderson


In [5]:

batter_versus_pitch_type.head()

,IDfg,wFB,wSL,wCT,wCB,wCH,wSF,wKN
0,15640,48.4,18.8,6.2,1.3,5.2,5.9,0.0
6,25764,28.1,20.3,4.1,1.3,-0.5,8.2,0.0
2,19755,31.7,14.2,1.1,9.2,8.2,8.2,0.2
3,20123,39.2,4.3,7.3,7.8,7.1,2.0,0.0
19,26289,8.5,13.3,7.1,8.7,5.5,0.5,0.0


In [6]:

batter_hit_stats.head()

,IDfg,AVG+,OBP+,BABIP+,SLG+,BB%+,HardHit%
0,15640,134,148,129,178,233,0.609
6,25764,138,126,124,149,99,0.481
2,19755,125,124,114,160,134,0.595
3,20123,120,136,104,144,223,0.568
19,26289,117,118,112,134,134,0.539
